# Table of Contents
 <p><div class="lev1 toc-item"><a href="#md_ipynb.rb" data-toc-modified-id="md_ipynb.rb-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>md_ipynb.rb</a></div><div class="lev1 toc-item"><a href="#convertの手順" data-toc-modified-id="convertの手順-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>convertの手順</a></div><div class="lev1 toc-item"><a href="#figs_cpの手順" data-toc-modified-id="figs_cpの手順-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>figs_cpの手順</a></div><div class="lev1 toc-item"><a href="#attach_viewをmdフォーマットに変換" data-toc-modified-id="attach_viewをmdフォーマットに変換-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>attach_viewをmdフォーマットに変換</a></div><div class="lev2 toc-item"><a href="#error" data-toc-modified-id="error-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>error</a></div>

# md_ipynb.rb


1. sourceから一行ずつ読み出して，'#'で切り出して，contへlinesの塊として保存していく．
   1. こうするとipynbで単元ごとの展開-縮小表示を切り替え可能．
1. md_cell0を作って，sourceのcontにあるlinesを入れる．
1. それをさらにipynb["cells"]に追加していく.
1. targetにjson形式でdump.

``` ruby
# md_ipynb.rb
# convert md to ipynb with splitted at '#'

require 'json'

source = File.read(ARGV[0])

cont = []
cont0 = ""
source.each_line do |line|
  if m=line.match(/^#/)
    cont << cont0
    cont << line
    cont0 = ""
  else
    cont0 += line
  end
end
cont << cont0

ipynb = JSON.load(File.read('template.ipynb'))

cont.each do |lines|
  md_cell0 = {"cell_type"=>"markdown", "metadata"=>{}, "source"=>[]}
  md_cell0["source"] << lines
  ipynb["cells"] << md_cell0
end

File.open('tmp.ipynb','w') do |target|
  JSON.dump(ipynb,target)
end
```


# convertの手順
hikiのtextをnotebooksに動かす手順．
* namesに含まれているnew_ist_dataにあるmapleテキストをhiki2mdで変換
* md_ipynb.rbでipynbに変換


In [1]:
name = ""
# 
  p hiki = File.join(ENV["HOME"],'Sites','new_ist_data/maple_hiki_data/text',
    name)
  p md = "md_sources/#{name.downcase}.md"
#  system "hiki2md #{hiki} > #{md}" # mdをいじっていくので，これをskip

  system "ruby md_ipynb.rb #{md}"
  p ipynb = File.join('tmp',name.downcase+'.ipynb')
  system "mv tmp.ipynb #{ipynb}"
end

"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/FSolve"
"md_sources/fsolve.md"
"notebooks/fsolve.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/Error"
"md_sources/error.md"
"notebooks/error.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/LAFundamentals"
"md_sources/lafundamentals.md"
"notebooks/lafundamentals.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/LAEigenvectors"
"md_sources/laeigenvectors.md"
"notebooks/laeigenvectors.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/LAMatrixInverse"
"md_sources/lamatrixinverse.md"
"notebooks/lamatrixinverse.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/InterpolationIntegral"
"md_sources/interpolationintegral.md"
"notebooks/interpolationintegral.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/LeastSquareFit"
"md_sources/leastsquarefit.md"
"notebooks/leastsquarefit.ipynb"
"/Users/bob/Sites/new_ist_data/maple_hiki_data/text/NonLinearFit"
"md_sources/nonlinearfit.md"
"noteboo

["FSolve", "Error", "LAFundamentals", "LAEigenvectors", "LAMatrixInverse", "InterpolationIntegral", "LeastSquareFit", "NonLinearFit", "FFT"]

# figs_cpの手順
hikiのcache/attachにあるpngをnotebooks/figsに動かす.



In [12]:
require 'systemu'
require 'fileutils'
names = ["FSolve","Error",
"LAFundamentals","LAEigenvectors","LAMatrixInverse",
"InterpolationIntegral","LeastSquareFit",
"NonLinearFit","FFT"]

names.each do |name|
  # 
  attach = File.join(ENV["HOME"],'Sites','new_ist_data/maple_hiki_data/cache/attach',
    name)

  status, stdout, stderr = systemu "ls #{attach}"
  files = stdout.split("\n")
  next if files == []
  files.each do |file|
    source = File.join(attach, file)
    target = File.join('../notebooks/figs', file)
    FileUtils.cp(source, target, :verbose => true)
  end
end

["FSolve", "Error", "LAFundamentals", "LAEigenvectors", "LAMatrixInverse", "InterpolationIntegral", "LeastSquareFit", "NonLinearFit", "FFT"]

# attach_viewをmdフォーマットに変換


In [4]:
["tmp0.ipynb"].each do |file|
  next unless file.match(/ipynb$/)
  p file
  lines = File.readlines(file)
  target = File.open('tmp.ipynb','w')
  lines.each do |line|
    if m = line.match(/attach_view\((.+?),/)
      p line = '    "!['+m[1]+'](figs/'+m[1]+')\n",'+"\n"
    end
    if m = line.match(/\"\|\:----\|\\n/)
      p line
      p line = nil
    end

    target.print(line) 
  end
  target.close 
end

"tmp0.ipynb"
"    \"![C8_LeastSquareFitplot2d1.png](figs/C8_LeastSquareFitplot2d1.png)\\n\",\n"
"    \"|:----|\\n\",\n"
nil
"    \"![C8_LeastSquareFitplot2d2.png](figs/C8_LeastSquareFitplot2d2.png)\\n\",\n"
"    \"|:----|\\n\",\n"
nil
"    \"![C8_LeastSquareFitplot3d3.png](figs/C8_LeastSquareFitplot3d3.png)\\n\",\n"
"    \"|:----|\\n\",\n"
nil
"    \"![C8_LeastSquareFitplot3d4.png](figs/C8_LeastSquareFitplot3d4.png)\\n\",\n"
"    \"|:----|\\n\",\n"
nil


["tmp0.ipynb"]

## error
json.loadのerrorが出る．

cellの最後で変換してしまうと，["...",]で終わり，そうすると読めない．codingに組み込むのはめんどいので，手で修正．